In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import time
import os
import copy
from torchvision import datasets, models, transforms

import torch.utils.data as utils
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import time
from torch.optim import lr_scheduler
from PIL import Image
import PIL.ImageOps    
import torch.nn.functional as F
import pandas as pd 

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.0.1
Torchvision Version:  0.2.1


In [2]:
class DepressingNet(nn.Module):
    def __init__(self, model):
        super(DepressingNet, self).__init__()

        # Setting up pretrained model
        self.model = model

        # Defining the fusion layers
        self.fusion_layer = nn.Sequential(
                                nn.Linear(4,4),
                                nn.ReLU(inplace=True),
                                nn.Linear(4,4),
                                nn.Linear(4,2))



    def forward(self, input1, input2):
        # forward pass of input 1
        output1 = self.model(input1)
        # forward pass of input 2
        output2 = self.model(input2)
        #applying fusion layer
        concat = torch.cat((output1, output2),1)
#         concat = concat.to('cpu')
        res = self.fusion_layer(concat)
        return res 

In [3]:
model_ft = models.vgg11_bn(pretrained=True)
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs,2)
model = DepressingNet(model_ft)

model.load_state_dict(torch.load('58_4_model.pt'))

In [4]:
imagesdf = pd.read_csv('1_depressing.csv') 
imagesdf.head()

,id,lat,long
0,50e5f7d4d7c3df413b00056d,22.314273,114.177176
1,50e5f7d4d7c3df413b00056e,22.332412,114.204790
2,50e5f7d4d7c3df413b00056f,22.316815,114.217950
3,50e5f7d5d7c3df413b000571,22.331419,114.149007
4,50e5f7d5d7c3df413b000572,22.298067,114.174612


In [5]:
transform = transforms.Compose([transforms.Resize((224, 224)),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                                                      ])
def imshow(img,text=None,should_save=False):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()    
    
def open_image(name):
    n = name+'.jpg'
    path = os.path.join('/home/bill/urban_planning/images_only/', str(n))
    image = Image.open(path)
    img = transform(image) 
    return img
    

In [6]:
depressingdf = pd.DataFrame(columns=['left','right','winner'])
cont = ['left', 'right']

In [7]:
import random
ind = 0
model=model.to('cuda:0')
for i in imagesdf['id']:
    sample = random.sample(list(imagesdf['id']),50)
    for j in sample:
        if i != j:
            img1 = open_image(i).unsqueeze(0)
            img2 = open_image(j).unsqueeze(0) 
            img1=img1.to('cuda:0')
            img2=img2.to('cuda:0')
            res = model(img1, img2)
            r = torch.argmax(res).item()
            depressingdf.loc[ind] = [i, j, cont[r]]
            ind+=1
            if ind%1000 == 0:
                print(ind)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [8]:
depressingdf.head()

,left,right,winner
0,50e5f7d4d7c3df413b00056d,50f43537fdc9f065f0002d0e,left
1,50e5f7d4d7c3df413b00056d,50f42c09fdc9f065f000175c,right
2,50e5f7d4d7c3df413b00056d,50f4352cfdc9f065f0002c2b,left
3,50e5f7d4d7c3df413b00056d,50e748e4d7c3df413b0013cb,right
4,50e5f7d4d7c3df413b00056d,50f447d2fdc9f065f0003a81,right


In [9]:
depressingdf.to_csv('1depscore.csv')